<a href="https://colab.research.google.com/github/ShalilG/Stock-Market-Analysis/blob/main/StockPredictionLinearRegression%26SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 1.0 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
#Import the libraries 

import yfinance as yf 
import pandas as pd 
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [3]:
#Load the data 
tick = 'AAPL'

#Show the data for Adj Close
df = yf.download(tick, start = '1987-05-18', end = '2022-8-9')
df = df[['Adj Close']]

df

[*********************100%***********************]  1 of 1 completed


,Adj Close
Date,
1987-05-18,0.263984
1987-05-19,0.255272
1987-05-20,0.259628
1987-05-21,0.259628
1987-05-22,0.258321
...,...
2022-08-02,159.788040
2022-08-03,165.899567
2022-08-04,165.580002


In [4]:
#A variable for predicting 'n' days out into the future 
forecast_out = 30

#Create another column (the target or dependent variable) shiften 'n' units up
#The prediction column is the following day's price close, used for training
df['Prediction'] = df[['Adj Close']].shift(-forecast_out)
#Print new dataset 

print(df.tail())

             Adj Close  Prediction
Date                              
2022-08-02  159.788040         NaN
2022-08-03  165.899567         NaN
2022-08-04  165.580002         NaN
2022-08-05  165.350006         NaN
2022-08-08  164.869995         NaN


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [5]:
#Create the independent data set (X)
#Convert the dataframe to a numpy array 

X = np.array(df.drop(['Prediction'],1))

#Remove the last 'n' rows 

X = X[:-forecast_out]
print(X)

[[  0.26398417]
 [  0.25527179]
 [  0.25962752]
 ...
 [135.16226196]
 [138.07820129]
 [141.46350098]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  after removing the cwd from sys.path.


In [6]:
#Create the dependent data set (y)
#Convert the dataframe to a numpy array (All of the values including NaN's)
y = np.array(df['Prediction'])

#Get all of the y values except the last 'n' rows 
y = y[:-forecast_out]
print(y)

[  0.28227973   0.27879506   0.28315091 ... 165.58000183 165.3500061
 164.86999512]


In [7]:
#Split the data into 80% training and 20 % testing
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [8]:
#Create and train the Support Vector Machine (Regressor)

svr_rbf = SVR(kernel='rbf', C=1e3, gamma=0.1)
svr_rbf.fit(x_train, y_train)

SVR(C=1000.0, gamma=0.1)

In [9]:
#Testing Model: Score returns the coefficient of determination R^2 of the prediction 
#The best possible score is 1.0
svm_confidence = svr_rbf.score(x_test, y_test)
print('svm confidence: ', svm_confidence)

svm confidence:  0.9892489431825975


In [10]:
#Create and train the Linear Regression Model 
lr = LinearRegression()

#Train the model
lr.fit(x_train, y_train)

LinearRegression()

In [11]:
#Testing Model: Score returns the coefficient of determination R^2 of the prediction 
#The best possible score is 1.0

lr_confidence = lr.score(x_test, y_test)
print('LR confidence: ', lr_confidence)

LR confidence:  0.9855477561071461


In [12]:
# Set x_forecast equal to the last 30 rows of the original data set from the Adj Close column 
x_forecast = np.array(df.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[141.46350098]
 [137.24935913]
 [139.03686523]
 [136.53034973]
 [138.73727417]
 [141.3636322 ]
 [142.72175598]
 [146.14700317]
 [146.83602905]
 [144.66903687]
 [145.65766907]
 [145.28819275]
 [148.26405334]
 [149.96170044]
 [146.86599731]
 [150.7905426 ]
 [152.82771301]
 [155.13452148]
 [153.87625122]
 [152.73783875]
 [151.38972473]
 [156.57250977]
 [157.13174438]
 [162.28457642]
 [161.28596497]
 [159.78804016]
 [165.89956665]
 [165.58000183]
 [165.3500061 ]
 [164.86999512]]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [13]:
#Print the Linear Regression predictions for the next 'n' days 
lr_prediction = lr.predict(x_forecast)
print(lr_prediction)

[144.536445   140.23581182 142.06000482 139.50204542 141.75426493
 144.43452651 145.82052452 149.31607172 150.01923921 147.80777153
 148.81669449 148.43963501 151.47657238 153.2090621  150.04982255
 154.05491547 156.13389691 158.48805048 157.20395539 156.04217806
 154.66639526 159.95555213 160.52626452 165.78485363 164.76574661
 163.2370783  169.47403818 169.14791459 168.91319841 168.42333565]


In [14]:
#Print the Support Vector Regressor predictions for the next 'n' days 
svm_prediction = svr_rbf.predict(x_forecast)
print(svm_prediction)

[147.28140963 142.53124061 150.08256215 133.04060699 150.39364799
 147.26694163 147.59429316 148.31619448 147.98312482 148.71775467
 148.71681989 148.86048431 153.37457981 158.01112844 147.99487797
 150.51052119 144.10632122 147.20684554 148.25183081 143.65269892
 144.61099556 151.29459838 153.95438379 166.09907014 168.64618637
 167.51486348 165.39839651 164.15090399 163.84730882 164.51874759]
